<a href="https://colab.research.google.com/github/Basma29/NLPTasks/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [ ]:
import pandas as pd
df = pd.read_csv("IMDB Dataset.csv", encoding="utf-8", sep=",", on_bad_lines="skip", quoting=3)

print(df.head())
print(df.columns)
print(f"Dataset has {df.shape[1]} columns")

                                                                                                      m I watch all of them. Horror especially. My friends and I all gather after a hard week at school and work  \
"During World War II                               two Byelorussian (Soviet Russian) soldiers try ...               as they trudge through snowy terrain                                                           
"Gilmore Girls is a hilarious show with never e... wit                                                  and charm. At age 16 Lorelai Gilmore gave bir...                                                           
"A found tape about 3? guys having fun torturin... spoiler.<br /><br />First of all                          the acting made this short not scary at all                                                           
"The Biggest one that bugs the hell out of me i... its like saying ""well he takes French commands...   even if its mend to be a comedy.<br /><br />T...

In [ ]:
df = df.iloc[:, [0, 1]]
df.columns = ["review", "sentiment"]
df.columns = df.columns.str.strip()
df = df.dropna()
print(f"Dataset size after cleaning: {df.shape}")


Dataset size after cleaning: (33691, 2)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
texts = df['review'].astype(str).tolist()
labels = df['sentiment'].apply(lambda x: 1 if x == "positive" else 0).values

x_train, x_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

vocab_size = 10000
max_length = 200
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post', truncating='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding='post', truncating='post')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=32, input_length=max_length),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=64, validation_data=(x_test_pad, y_test))


Epoch 1/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 53s 116ms/step - accuracy: 0.9412 - loss: 0.2218 - val_accuracy: 0.9549 - val_loss: 0.1842
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 78s 107ms/step - accuracy: 0.9592 - loss: 0.1705 - val_accuracy: 0.9549 - val_loss: 0.1839
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 83s 110ms/step - accuracy: 0.9557 - loss: 0.1814 - val_accuracy: 0.9549 - val_loss: 0.1838
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 83s 113ms/step - accuracy: 0.9559 - loss: 0.1811 - val_accuracy: 0.9549 - val_loss: 0.1848
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 80s 107ms/step - accuracy: 0.9554 - loss: 0.1832 - val_accuracy: 0.9547 - val_loss: 0.1838


In [ ]:
test_loss, test_acc = model.evaluate(x_test_pad, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

211/211 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9566 - loss: 0.1785
Test Accuracy: 95.47%
